In [1]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276
#3) https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?Expires=1585601065&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=Lnpf6wT8rkozSh9av7U9nGuC7WAH6KuI2Cj3Y7G366gkGlh8D-Ie1Kc~TyBAUu~uMsVltleJcSv3p6TCm6HdFnhpyoTgLcYh6eFfvQwIUqbk1Bf4JZldgB~BDKUOwY1G0pA-HoKjvIAu3avO98SMO35upakm9OEBByd4nC9aXsjKRThd6bTpq1qIuuD9gh1l5FaM6hNRB0c2lCf4Q3adx7C3FW0NMwdWhcuF45A9f~dO3zTWWSQamoo5Otc-PHMMt96TetNcML~jy9ghgJeCPY6DJLUIwQAt03fENBluS~TjTJ17WD~n51xiRofb94fEJHoRHh0d-430LLwr7BX4IA__

In [538]:
import numpy as np
import os
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import zscore
from scipy.stats import kurtosis, skew
from scipy.signal import lfilter
import librosa
import pysptk
import math
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier


#settings
import config

## Reading recordings

In [462]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [511]:
#Read wav data set

if config.featExtr_skip is False:
    #print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        #print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

data/YT_set/edited_wavs/edit_Wet Throat Infection Cough-tfc5cXiXMDc-Wet.wav
data/YT_set/edited_wavs/edit_Wet Throat Infection Cough-tfc5cXiXMDc-Wet.wav
data/YT_set/edited_wavs/edit_Heavy cold and sore throat coughing.-NaOVmYoIjbs-Dry.wav
data/YT_set/edited_wavs/edit_Heavy cold and sore throat coughing.-NaOVmYoIjbs-Dry.wav
data/YT_set/edited_wavs/edit_Spring Cold Coughing 2-AQOeIVbhFm4-Dry.wav
data/YT_set/edited_wavs/edit_Spring Cold Coughing 2-AQOeIVbhFm4-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_# 34 coughing up crap again-rkF_uMizqoc-Wet.wav
data/YT_set/edited_wavs/edit_# 34 coughing up crap again-rkF_uMizqoc-Wet.wav
data/YT_set/edited_wavs/edit_# 30 Chesty and wet cough-d2wkdrScerU-Wet.w

Listening to some of the audios

In [512]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Dry')

In [513]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Wet')

In [514]:
if config.featExtr_skip is False:
    s=all_s[15]
    s

In [522]:
#compute RMS value of a signal and return it (in dB scale)
#seems not to work?
def get_RMS(s):
    s_rms = np.sqrt(np.mean(np.power(s,2)))
    #convert to dB scale
    #s_db = 20*np.log10(s_rms/1.0)
    return s_rms

#seems not to work either
#RMS-based normalization of a signal, based on a target level (in dB)
def RMS_normalization(s, dB_targ):
    
    #desired level is converted to linear scale
    rms_targ = 10**(dB_targ/20.0)
    
    #compute scaling factor
    scale = rms_targ/get_RMS(s)
    
    #scale amplitude of input signal
    scaled_s = scale*s
    
    return scaled_s

def match_target_amplitude(audioSegment_sound, target_dBFS):
    dBFS_diff = target_dBFS - audioSegment_sound.dBFS
    return audioSegment_sound.apply_gain(dBFS_diff)

#Apply pre-emphasis (high-pass) filter
def apply_preEmph(x):
    x_filt = lfilter([1., -0.97], 1, x)
    return x_filt
        
#Obtain autocorrelation
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[int((result.size+1)/2):] #Note: other people use re.size/2:, but this does not work for me 
                                   # TODO: check consistency in other computers

#Compute zero-crossing rate
def get_zcr(x):
    zcr = (((x[:-1] * x[1:]) < 0).sum())/(len(x)-1)
    return zcr

#Compute log-energy
def get_logEnergy(x):
    logEnergy = np.log10( ( (np.power(x,2)).sum()/len(x) ) + config.eps)  
    return logEnergy

#Estimate fundamental frequency (F0)
def get_F0(x,fs):
    #autocorrelation-based method to extract F0
    xcorr_arr = autocorr(x)
    
    #looking for F0 in the frequency interval 50-500Hz, but we search in time domain
    min_ms = round(fs/500)
    max_ms = round(fs/50)
    
    xcorr_slot = xcorr_arr[max_ms+1:2*max_ms+1]
    xcorr_slot = xcorr_slot[min_ms:max_ms]
    t0 = np.argmax(xcorr_slot)
    F0 = fs/(min_ms+t0-1)
    return F0

#Estimate formants
def get_formants(x, lp_order, nr_formants):
    
    #compute lp coefficients
    a = librosa.lpc(x, lp_ord)
    

    #get roots from lp coefficients
    rts = np.roots(a)
    rts = [r for r in rts if np.imag(r) >= 0]

    #get angles
    angz = np.arctan2(np.imag(rts), np.real(rts))

    #get formant frequencies
    formants = sorted(angz * (fs_targ / (2 * math.pi)))
    
    return formants[0:nr_formants]

def get_entropy(x, type='shannon'):
    #default shannon entropy since this is the one used by the phd thesis
    
    base = {'shannon' : 2., 'natural' : math.exp(1), 'hartley' : 10.}
    N = len(x)

    if N <= 1:
        return 0

    value,counts = np.unique(x, return_counts=True)
    probs = counts / N
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0. #initialization

    # compute entropy
    for i in probs:
        ent -= i * math.log(i+config.eps, base[type])
    
    return ent


In [627]:
def feature_extraction(x,fs,feats_df,lp_ord,ID,label):
#Extract features from signal x (identified as ID), and concatenate them to dataframe feats_df
#Features' reference: (see Appendix)
#[1]https://link.springer.com/article/10.1007/s10439-013-0741-6
#[2]https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?dsi_version=c5434db897ab74b192ca295a9eeca041&Expires=1585086202&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=c8k8DmG~KIxg0ToTO8rebm2MzHneCzJGkjSFRB7BYTEQ-MHXEr0ocHmISrldP3hFf9qmeiL11ezyefcNeRVeKIQ9PVjOl9pn7rXWcjA1o2voPn1VnDd8n7G2cT31apdj0LNMclhlXRPnCsGD66qDRqa3d-xaqqXhEqU73aw3ZgBgroO213MfJOqFhJxxXo2QEia0bSlDRTeX9KhSczFK-IFTPC6GwFL2L04por8pQRI3HF7E3f26O9zp9OhkwxSU9qfJah20WxZLA4PxREdv7JGoVBinR6T0mTcIaQi~B4IzYjSPSsTTADMNk5znVYIvSqgtMT~DY~qwlfq4SRdFjQ__
  
    
    #do features in a frame-basis
    x_frames = spe_feats.sigproc.framesig(x,config.frame_len,config.frame_step,config.win_func) #DOUBT: should I use window or not?
                                                                        #at least for formant estimation i should

    nr_frames = x_frames.shape[0]
    #print(nr_frames)
        
    #0)Wavelets #TODO
    
    #DOUBT: if log-energy feature is included, should I also include the first mfcc coefficient (c0) ?
    #1)mfcc
    mfcc_feat = spe_feats.mfcc(x,fs, winlen=config.frame_len_s,winstep=config.frame_step_s,    numcep=config.cep_num,winfunc=config.win_func)
   # print('MFCC Features --- ', mfcc_feat)
    mfcc_feat = zscore(mfcc_feat, axis=1,ddof=1)
  #  print('MFCC Features new --- ', mfcc_feat)
    #deltas to capture 
    mfcc_delta_feat = spe_feats.delta(mfcc_feat,1) #mfcc_delta_feat = np.subtract(mfcc_feat[:-1], mfcc_feat[1:]) #same
    mfcc_deltadelta_feat = spe_feats.delta(mfcc_delta_feat,1)          
    
    #2)zero-crossing rate
    zcr_feat = np.apply_along_axis(get_zcr, 1, x_frames)
    
    #3)Formant frequencies
    #using LP-coeffcs-based method
    #formant_feat = np.apply_along_axis(get_formants, 1, x_frames, lp_ord, nr_formants)
    
    #Note: for the moment, it seems some frames are ill-conditioned for lp computing,
    #current solution - we skip those and fill with NaN values
    formants_feat= np.empty((nr_frames,4))
    formants_feat[:] = np.nan
    
    for i_frame in range(0,nr_frames):
        try: 
            formants_feat[i_frame] = get_formants(x_frames[i_frame], config.lp_ord, config.nr_formants)
        except:
            pass
    
    #4)Log-energy
    logEnergy_feat =  np.apply_along_axis(get_logEnergy, 1, x_frames)
    
    #5)Pitch (F0)
    F0_feat =  np.apply_along_axis(get_F0, 1, x_frames,fs)
    
    #TODO: compute also F0 with pysptk (a python wrapper for SPTK library), it probably gives better results
    #https://github.com/r9y9/pysptk/blob/master
        	#F0_feat = pysptk.rapt(x.astype(np.float32), fs=fs, hopsize=frame_step, min=50, max=500, ,voice_bias=0.0 ,otype=\"f0\")
    
    #Compare the values between swipe and rapt 
    #F0_feat = pysptk.swipe(x.astype(np.float64), fs=fs,hopsize = config.frame_step, min=50, max=500, otype="f0")
    
    F0_feat = pysptk.rapt(x.astype(np.float32), fs=fs,hopsize = config.frame_step, min=50, max=500, otype="f0")

    
    
    #right frame size???
#Change the window size from 450 to 40 to 100
# Keep swipe , change min to 50 and max - 500
#EXample pysptk.swipe(x.astype(np.float64), fs=fs, hopsize=80, min=60, max=200, otype="f0")

    
    #6)Kurtosis
    kurt_feat =  np.apply_along_axis(kurtosis, 1, x_frames)
    
    #7)Bispectrum Score (BGS)
    #TODO: see PhD thesis for more info on this feature
    
    #8)Non-Gaussianity Score (NGS)
    #TODO: see PhD thesis for more info on this feature
   
    #9) Adding skewness as measure of non-gaussianity (not in paper)
    skew_feat =  np.apply_along_axis(skew, 1, x_frames)
    
    #DOUBT: 10) Shannon entropy GETTING -inf in all cases, WHY??? Don't include until fixed
    entropy_feat = np.apply_along_axis(get_entropy, 1, x_frames)

    
    #TODO: add small value in all entries, this may fix the problem
    
    mfcc_cols = ['mfcc_%s' % s for s in range(0,config.cep_num)]
    mfcc_delta_cols = ['mfcc_d%s' % s for s in range(0,config.cep_num)]
    mfcc_deltadelta_cols = ['mfcc_dd%s' % s for s in range(0,config.cep_num)]
    formants_cols = ['F%s' % s for s in range(1,config.nr_formants+1)]
          
    feats_segment = pd.concat([pd.DataFrame({'Id': ID, 'kurt': kurt_feat, 'logEnergy': logEnergy_feat,
                                                 'zcr': zcr_feat, 'F0': F0_feat,
                                                 'skewness': skew_feat, 'label': label, 'entropy':entropy_feat}),
                               pd.DataFrame(mfcc_feat,columns=mfcc_cols), 
                               pd.DataFrame(mfcc_delta_feat,columns=mfcc_delta_cols), 
                               pd.DataFrame(mfcc_deltadelta_feat,columns=mfcc_deltadelta_cols), 
                            pd.DataFrame(formants_feat,columns=formants_cols)],axis=1)
    
    #print(nr_frames)
    feats_df = feats_df.append(feats_segment,ignore_index=True, sort=False)
    
    return feats_df


In [628]:
def feature_extraction_Step(all_s,all_id,all_label):
    
    import config
    import pydub
    
    #initialize data frame of features:
    feats = pd.DataFrame([])

    for s, ID, label in zip(all_s,all_id,all_label):

            #Pre-processing of the signals:

            ## 0 ) Resampling to target sampling frequency:
            s = s.set_frame_rate(config.fs_targ)
            fs= config.fs_targ

            ## 1)
            if config.norm_skip is False:
                s=match_target_amplitude(s, config.dB_targ)
                #print(s.rms)

            ## 2) Segmentation of cough streams (silence-based)
            #min_silence_len in ms, silence_thresh in dB
            s_segments = split_on_silence (s, min_silence_len = 600, silence_thresh =s.dBFS-10)

            #checks that segmentation and removal of silence is OK
            #print(len(s_segments))
            #play(s)
            #input("Press Enter to continue...")               
            #for i in range(len(s_segments)):
            #    play(s_segments[i])
            #    input("Press Enter to continue...")               

            ## 3) Convert s_segments to numpy array format
            AudioSegment2numpy_arr = lambda x: np.asarray(x.get_array_of_samples())
            s_segments_np = list(map(AudioSegment2numpy_arr, s_segments))


            ## 4) Pre-emphasis filtering on each segment
            if config.HPF_skip is False:
               # print('High-pass filtering...')       
                preEmph_filtering = lambda x: apply_preEmph(x)
                s_segments_filt = list(map(preEmph_filtering, s_segments_np))
            else:
                s_segments_filt = s_segments_np

            #print('Computing features...')
            #Feature extraction for each segment

            #(lambda function doesn't work )
            #feat_extr_step = lambda x, fs, feats_df, lp_ord, ID: feature_extraction(x,fs,feats_df,lp_ord,ID)
            #feats = feat_extr_step(s_segments_filt,fs,feats,lp_ord,ID)
            for idx, seg_i in enumerate(s_segments_filt):
                #print('\tSegment %d' % idx)
                #print("feats --- ", feats)
                feats = feature_extraction(seg_i,fs,feats,config.lp_ord,ID,label)
                
    return feats

## Feature extraction

In [629]:
#import featureExtractionFunctions as feat

if config.featExtr_skip is False:

    feats = feature_extraction_Step(all_s,all_id,all_label)
    print(len(feats))
    
    
       

13410


## Load  (or store) features 

In [630]:

feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [631]:
#1.Check which columns have NaNs values

#feats2 = feats.copy()

#sum(feats.isna().any())
#feats.columns[feats.isna().any()].tolist() --> We get just the ones we have inserted in formants
feats2 = feats.interpolate(method ='pad')
#feats2 = feats.dropna(axis=1).copy()
#feats2.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)

#feats2.columns[feats2.isna().any()].tolist()

#feats2 = processingNaNvalues(feats)
feats2.describe()
#sum(feats2.isna().any())


,kurt,logEnergy,zcr,F0,skewness,entropy,mfcc_0,mfcc_1,mfcc_2,mfcc_3,...,mfcc_dd7,mfcc_dd8,mfcc_dd9,mfcc_dd10,mfcc_dd11,mfcc_dd12,F1,F2,F3,F4
count,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,...,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000,13410.000000
mean,3.709398,4.258418,0.681179,30.297604,-0.008360,8.568503,1.309688,-2.144157,0.189047,-0.589077,...,0.000205,-0.000216,-0.000187,-0.000862,-0.000493,-0.000282,-0.000282,-0.000282,-0.000282,-0.000282
std,3.851946,2.183823,0.165611,73.206673,0.345128,0.455845,0.366096,0.601133,0.482312,0.567827,...,0.236534,0.246343,0.233121,0.230055,0.236358,0.225308,0.225308,0.225308,0.225308,0.225308
min,-0.310083,-2.388365,0.012531,0.000000,-2.557286,0.402292,0.098600,-3.255308,-1.056984,-2.244664,...,-0.970065,-0.978169,-1.023406,-0.993991,-1.438780,-0.849354,-0.849354,-0.849354,-0.849354,-0.849354
25%,1.582898,2.329294,0.503759,0.000000,-0.116626,8.643856,1.066769,-2.564830,-0.148222,-1.010794,...,-0.146204,-0.157349,-0.154346,-0.147343,-0.152147,-0.146656,-0.146656,-0.146656,-0.146656,-0.146656
50%,2.466519,5.100722,0.726817,0.000000,-0.005592,8.643856,1.297403,-2.198181,0.153556,-0.626465,...,-0.001471,-0.002049,-0.007214,-0.004467,-0.000214,-0.000237,-0.000237,-0.000237,-0.000237,-0.000237
75%,5.056754,5.980990,0.817043,0.000000,0.102488,8.643856,1.531373,-1.770850,0.517530,-0.195410,...,0.142314,0.154373,0.146643,0.147291,0.150341,0.141532,0.141532,0.141532,0.141532,0.141532
max,86.069435,7.442851,0.987469,472.705536,5.977376,8.643856,2.666019,0.544302,1.613358,1.450469,...,1.230357,1.211892,0.951538,0.944678,1.095596,1.052831,1.052831,1.052831,1.052831,1.052831


In [632]:
 
#Make dictionary and add label column using it 
def addLabel2df(feats):
    feats_unique = feats.drop_duplicates(subset=['Id'])
    label_dict = dict(zip(feats_unique.Id, feats_unique.label))
    return label_dict


def frame_mean_std_chunk_modeling (feats2, label_dict):
    
    #Grouping the frames from a same recording (Id) into chunks with the same number of frames.
    #The training of the classifier will be based on these chunks mean and standard deviation.

    feats2['cum_IDidx'] = feats2.groupby('Id').cumcount()

    def get_subidx(cum_Idx,batch_size):
        #batch needs to be an integer (or float like 3.0)
        return int(1.0*cum_Idx/batch_size)

    feats2['subIdx'] = feats2.apply(lambda x: get_subidx(x['cum_IDidx'], 10), axis=1)
    feats2 = feats2.drop(['cum_IDidx'],axis=1)
    
    mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
    std_feats = feats2.groupby(['Id','subIdx']).agg(lambda x: x.std(ddof=0)).reset_index() #ddof=0 to compute population std (rather than sample std)
    keep_same = {'Id', 'subIdx'}
    mean_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_m') for c in mean_feats.columns]
    std_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_std') for c in std_feats.columns]
    
    mean_std_feats = pd.merge(mean_feats, std_feats, on=['Id','subIdx'], how='outer')
    
    mean_std_feats['label'] = mean_std_feats["Id"].map(label_dict)
    #mean_std_feats[['Id','label']].head(50)
    
    return mean_std_feats
   

#TODO: modeling of chunks using sequence models too

label_dict = addLabel2df(feats2)
mean_std_feats = frame_mean_std_chunk_modeling (feats2,label_dict)


print(mean_std_feats)

#sum(mean_std_feats.isna().any())
#mean_std_feats.describe()

               Id  subIdx    kurt_m  logEnergy_m     zcr_m        F0_m  \
0     1UDFq2InljM       0  2.893929     1.531556  0.585213    0.000000   
1     1UDFq2InljM       1  2.110001     5.906655  0.483208    0.000000   
2     1UDFq2InljM       2  1.625414     5.475906  0.387469  180.043884   
3     1UDFq2InljM       3  4.231469     4.748404  0.575439    0.000000   
4     1UDFq2InljM       4  2.924504     2.394540  0.387719    0.000000   
...           ...     ...       ...          ...       ...         ...   
1351  zjd4HrJbc8o      26  1.422714     5.241664  0.815789  175.066574   
1352  zjd4HrJbc8o      27  3.486760     4.458311  0.779699  204.254715   
1353  zjd4HrJbc8o      28  3.801287     4.955278  0.698496    0.000000   
1354  zjd4HrJbc8o      29  3.711125     4.036131  0.659398   58.807899   
1355  zjd4HrJbc8o      30  2.353641     3.797773  0.801587  142.319473   

      skewness_m  entropy_m  mfcc_0_m  mfcc_1_m  ...  mfcc_dd8_std  \
0       0.097264   8.637356  1.295564 -2.

In [633]:
mean_std_feats.columns

Index(['Id', 'subIdx', 'kurt_m', 'logEnergy_m', 'zcr_m', 'F0_m', 'skewness_m',
       'entropy_m', 'mfcc_0_m', 'mfcc_1_m',
       ...
       'mfcc_dd8_std', 'mfcc_dd9_std', 'mfcc_dd10_std', 'mfcc_dd11_std',
       'mfcc_dd12_std', 'F1_std', 'F2_std', 'F3_std', 'F4_std', 'label'],
      dtype='object', length=101)

In [634]:
mean_std_feats.shape

(1356, 101)

In [665]:
#2. Get feature set, labels, and recording IDs
#X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
X_train = mean_std_feats.iloc[:,8:21]
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates().tolist()

ID_train.size
#ID_list.size
X_train.head(1)


,mfcc_0_m,mfcc_1_m,mfcc_2_m,mfcc_3_m,mfcc_4_m,mfcc_5_m,mfcc_6_m,mfcc_7_m,mfcc_8_m,mfcc_9_m,mfcc_10_m,mfcc_11_m,mfcc_12_m
0,1.295564,-2.423573,0.142089,-0.342158,-0.296728,0.188851,-0.005153,-0.080217,0.081012,-0.160599,0.060458,0.642605,0.897848


## Model training

In [666]:
pred_probs = pd.DataFrame([])

#leave-one-out based on recordings (we leave out one recording as test at a time)
for i in range(0,len(ID_list)):
    
    idnow = ID_list[i]
    ID_train_list = ID_train.to_list()
    val_index = [i for i, x in enumerate(ID_train_list) if x == idnow]
    train_index = [i for i, x in enumerate(ID_train_list) if x != idnow]

    X_train1, X_val1 = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train1, y_val1 = y_train.iloc[train_index], y_train.iloc[val_index]


    #normalize train set
    scaler = StandardScaler()
    scaler.fit(X_train1)
    X_trainNorm1 = scaler.transform(X_train1.values)
    X_valNorm1 = scaler.transform(X_val1.values)

    #TODO: optimize the penaly weight
    #https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html
    logReg = SGDClassifier(loss='log', penalty='elasticnet')
    logReg.fit(X_trainNorm1, y_train1)
    y_hat_prob = logReg.predict_proba(X_valNorm1)
    classes =logReg.classes_
    pred_probs = pred_probs.append(pd.DataFrame({'ID': ID_train[val_index], str(classes[0]): y_hat_prob[:,0],
                                                 str(classes[1]): y_hat_prob[:,1]}),ignore_index=True, sort=False)    


In [615]:
def predict_class(prob_dry,prob_wet):
    if prob_dry > prob_wet :
        return 'Dry'
    else:
        return 'Wet'

#get probability per recording
def get_predClass_per_audio(pred_probs, label_dict):

    mean_pred_probs = pred_probs.groupby('ID').aggregate('mean').reset_index()

    mean_pred_probs['pred_class'] = mean_pred_probs.apply(lambda x: predict_class(x['Dry'], x['Wet']), axis=1)
    
    #add actual classes
    mean_pred_probs['label'] = mean_pred_probs["ID"].map(label_dict)
    return mean_pred_probs

mean_pred_probs = get_predClass_per_audio(pred_probs, label_dict)

## Evaluation

In [616]:
import classifEvaluationFunctions as eval
eval.evaluation_Step(mean_pred_probs)
    

Scores:
Accuracy: 0.638889
Precision: 0.626623
F1-score: 0.624699
Recall: 0.623810

Confusion matrix
pred_class  Dry  Wet  All
label                    
Dry           8    7   15
Wet           6   15   21
All          14   22   36


In [657]:
#Model training using NN

In [667]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

X = X_trainNorm1
X.shape
number=LabelEncoder()
y = number.fit_transform(y_train1.astype('string'))
print(y.shape)
print(X.shape)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 127)


(1325,)
(1325, 13)


In [668]:
num_labels = 2
filter_size = 2
def build_model_graph():
    # define the keras model
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    #model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

model = build_model_graph()


# Display model architecture summary 
#model.summary()
# Calculate pre-training accuracy 
score = model.evaluate(x_test,y_test, verbose=0)
accuracy = 100*score[1]

In [669]:

from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
num_epochs = 200
num_batch_size = 30
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=0)

score = model.evaluate(x_test, y_test, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 82.91%


In [672]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 127)

In [673]:
accuracy=[]
for i in range(0,10):
    model = build_model_graph()
    num_epochs = 100
    num_batch_size = 30
    model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=0)
    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy.append(score[1])
    print("Training Accuracy: {0:.2%}".format(score[1]))

mean_accuracy = np.mean(accuracy)

print("Accuracy: {0:.2%}".format(mean_accuracy))   

Training Accuracy: 85.71%
Training Accuracy: 84.96%
Training Accuracy: 87.22%
Training Accuracy: 84.96%
Training Accuracy: 84.21%
Training Accuracy: 83.46%
Training Accuracy: 81.95%
Training Accuracy: 81.95%
Training Accuracy: 83.46%
Training Accuracy: 81.95%
Accuracy: 83.98%
